RBF

In [18]:
from filtrar_datos import filtrar_datos_en_rango
from procesar_tableros import procesar_tableros
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np
import scipy
import scipy.io
from procesar_tableros import *
from filtrar_datos import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import xlsxwriter
class RBFNetwork:
    def __init__(self, num_centers, sigma):
        self.num_centers = num_centers
        self.sigma = sigma
        self.centers = None
        self.weights = None

    def _calculate_activations(self, X):
        distances = scipy.spatial.distance.cdist(X, self.centers)
        activations = np.exp(-distances**2 / (2 * self.sigma**2))
        return activations

    def fit(self, X, y):
        indices = np.random.choice(X.shape[0], self.num_centers, replace=False)
        self.centers = X[indices]
        activations = self._calculate_activations(X)
        self.weights = np.linalg.pinv(activations.T @ activations) @ activations.T @ y

    def predict(self, X):
        activations = self._calculate_activations(X)
        return activations @ self.weights


Cargar datos

In [19]:
# Cargar datos
print("Cargando datos...")
file_path_1bit = "./10x10/10x10/1bit/DATASET_v7.mat"
file_path_2bit = "./10x10/10x10/2bit/DATASET_v7.mat"

mat_data1b = scipy.io.loadmat(file_path_1bit)
mat_data2b = scipy.io.loadmat(file_path_2bit)
print("Datos cargados.")

# Obtenemos los dataset
dataset1b = mat_data1b['DATASET']
dataset2b = mat_data2b['DATASET']

# Acceder a las variables dentro del archivo (1bit)
labels_1b = dataset1b['labels']
inputs_1b = dataset1b['input']
bits_1b = dataset1b['bits']
beams_1b = dataset1b['beams']

# Acceder a las variables dentro del archivo (2bit)
labels_2b = dataset2b['labels']
inputs_2b = dataset2b['input']
bits_2b = dataset2b['bits']
beams_2b = dataset2b['beams']

Cargando datos...
Datos cargados.


Filtrar datos y procesar tableros

In [20]:
# Filtrar datos de 1 bit
print("Filtrando datos de 1 bit...")
datos1b_filtrados_input, datos1b_filtrados_labels = filtrar_datos_en_rango(inputs_1b, labels_1b)

# Filtrar datos de 2 bits
print("Filtrando datos de 2 bits...")
datos2b_filtrados_input, datos2b_filtrados_labels = filtrar_datos_en_rango(inputs_2b, labels_2b)

print("Datos filtrados listos.")

# Procesar los tableros
print("Procesando tableros...")

# 1 bit
tableros_procesados_1b = procesar_tableros(datos1b_filtrados_labels, 1)

# 2 bit
tableros_procesados_2b = procesar_tableros(datos2b_filtrados_labels, 2)

# Verificar dimensiones antes de transponer
print("Dimensiones antes de reshaping:")
print("tableros_procesados_1b.shape:", tableros_procesados_1b.shape)
print("tableros_procesados_2b.shape:", tableros_procesados_2b.shape)

# Reshape tableros
rs_tableros_procesados_1b = tableros_procesados_1b.reshape(-1, tableros_procesados_1b.shape[1] * tableros_procesados_1b.shape[2])
rs_tableros_procesados_2b = tableros_procesados_2b.reshape(-1, tableros_procesados_2b.shape[1] * tableros_procesados_2b.shape[2])

print("Dimensiones después de reshaping:")
print("rs_tableros_procesados_1b.shape:", rs_tableros_procesados_1b.shape)
print("rs_tableros_procesados_2b.shape:", rs_tableros_procesados_2b.shape)

# Ajustar las dimensiones de datos1b_filtrados_labels
reshaped_labels_1b = datos1b_filtrados_labels.reshape(-1, datos1b_filtrados_labels.shape[0] * datos1b_filtrados_labels.shape[1])
reshaped_labels_2b = datos2b_filtrados_labels.reshape(-1, datos2b_filtrados_labels.shape[0] * datos2b_filtrados_labels.shape[1])

# Comprobar las dimensiones después del reshaping
print("Dimensiones de reshaped_labels_1b:", reshaped_labels_1b.shape)
print("Dimensiones de reshaped_labels_2b:", reshaped_labels_2b.shape)

# Comprobar las dimensiones de datos1b
print("Dimensiones de datos1b_filtrados_labels:", datos1b_filtrados_labels.shape)
print("Dimensiones de datos2b_filtrados_labels:", datos2b_filtrados_labels.shape)



Filtrando datos de 1 bit...
Filtrando datos de 2 bits...
Datos filtrados listos.
Procesando tableros...
Dimensiones antes de reshaping:
tableros_procesados_1b.shape: (9180, 10, 10)
tableros_procesados_2b.shape: (9180, 10, 10)
Dimensiones después de reshaping:
rs_tableros_procesados_1b.shape: (9180, 100)
rs_tableros_procesados_2b.shape: (9180, 100)
Dimensiones de reshaped_labels_1b: (9180, 100)
Dimensiones de reshaped_labels_2b: (9180, 100)
Dimensiones de datos1b_filtrados_labels: (10, 10, 9180)
Dimensiones de datos2b_filtrados_labels: (10, 10, 9180)


Pruebas

In [21]:
# Inicializar matrices para almacenar métricas
predicciones_100_RBFs = np.zeros((10, 10, rs_tableros_procesados_1b.shape[0]))
mse_metrics = np.zeros((10, 10))
mae_metrics = np.zeros((10, 10))
r2_metrics = np.zeros((10, 10))

print(f"rs_tableros_procesados_1b:  ", rs_tableros_procesados_1b.shape)

# Definir valores de alpha y sigma para las pruebas
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
sigmas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Realizar pruebas variando num_centers y sigma
for i in range(10):
    for j in range(10):
        y_labels = reshaped_labels_1b[:, i * 10 + j].reshape(-1, 1)
        print(f"Y_labels: ", y_labels.shape)
        X_train1, X_test1, y_train1, y_test1 = train_test_split(
            rs_tableros_procesados_1b, y_labels, test_size=0.4, random_state=42
        )
        print(f"Datos divididos para la celda ({i},{j}).")

        # Inicializar matriz para almacenar resultados
        results = np.zeros((len(alphas), len(sigmas)))

        for alpha_index, alpha in enumerate(alphas):
            num_centers = int(np.ceil(len(X_train1) * alpha))
            for sigma_index, sigma in enumerate(sigmas):
                #print(f"Probando con num_centers = {num_centers} y sigma = {sigma}...")
                rbf_net = RBFNetwork(num_centers=num_centers, sigma=sigma)
                rbf_net.fit(X_train1, y_train1)
                predictions_test = rbf_net.predict(X_test1)
                mse_test = mean_squared_error(y_test1, predictions_test)
                results[alpha_index, sigma_index] = mse_test
                #print(f"MSE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mse_test}")

        # Almacenar los resultados de la mejor configuración en las matrices de métricas
        best_alpha_index, best_sigma_index = np.unravel_index(np.argmin(results), results.shape)
        best_num_centers = int(np.ceil(len(X_train1) * alphas[best_alpha_index]))
        best_sigma = sigmas[best_sigma_index]

        # Entrenar la RBF con los mejores hiperparámetros
        rbf_net = RBFNetwork(num_centers=best_num_centers, sigma=best_sigma)
        rbf_net.fit(X_train1, y_train1)
        predictions_all = rbf_net.predict(X_test1)

        # Almacenar las predicciones y métricas
        predicciones_100_RBFs[i, j, :len(predictions_all)] = predictions_all.flatten()
        mse_metrics[i, j] = mean_squared_error(y_test1, predictions_all)
        mae_metrics[i, j] = mean_absolute_error(y_test1, predictions_all)
        r2_metrics[i, j] = r2_score(y_test1, predictions_all)




rs_tableros_procesados_1b:   (9180, 100)
Y_labels:  (9180, 1)
Datos divididos para la celda (0,0).
Probando con num_centers = 551 y sigma = 0.2...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.2: 0.4626811020746589
Probando con num_centers = 551 y sigma = 0.3...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.3: 0.4594382605157341
Probando con num_centers = 551 y sigma = 0.4...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.4: 0.4622262215959052
Probando con num_centers = 551 y sigma = 0.5...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.5: 0.45806110741581013
Probando con num_centers = 551 y sigma = 0.6...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.6: 0.4575354940895691
Probando con num_centers = 551 y sigma = 0.7...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.7: 0.4514673802572625
Probando con num_centers = 551 y sigma = 0.8...
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.8: 0.4509305

Guardar datos

In [ ]:
# Create the 'img' directory if it doesn't exist
if not os.path.exists('img'):
    os.makedirs('img')

# Placeholder for your metrics data
# Ensure mse_metrics, mae_metrics, and r2_metrics are numpy arrays of shape (10, 10)
mse_metrics = np.random.rand(10, 10)  # Replace with actual data
mae_metrics = np.random.rand(10, 10)  # Replace with actual data
r2_metrics = np.random.rand(10, 10)   # Replace with actual data

# Save results to an Excel file
workbook = xlsxwriter.Workbook('resultados_rbf3.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0, 0, 'Celda')
worksheet.write(0, 1, 'MSE')
worksheet.write(0, 2, 'MAE')
worksheet.write(0, 3, 'R2')

for i in range(10):
    for j in range(10):
        worksheet.write(i * 10 + j + 1, 0, f'({i},{j})')
        worksheet.write(i * 10 + j + 1, 1, mse_metrics[i, j])
        worksheet.write(i * 10 + j + 1, 2, mae_metrics[i, j])
        worksheet.write(i * 10 + j + 1, 3, r2_metrics[i, j])

workbook.close()

# Plot 3D surface for MSE
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(range(10), range(10))
ax.plot_surface(X, Y, cp.asnumpy(mse_metrics), cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('MSE')
plt.title('Superficie de MSE por celda')
plt.savefig('img/mse_surface.png')
plt.close(fig)

# Plot 3D surface for MAE
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, mae_metrics, cmap='plasma')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('MAE')
plt.title('Superficie de MAE por celda')
plt.savefig('img/mae_surface.png')
plt.close(fig)

# Plot 3D surface for R2
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, r2_metrics, cmap='inferno')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('R2')
plt.title('Superficie de R2 por celda')
plt.savefig('img/r2_surface.png')
plt.close(fig)

# Display results
print("Resultados de las métricas:")
for i in range(10):
    for j in range(10):
        print(f"Celda ({i},{j}): MSE={mse_metrics[i, j]}, MAE={mae_metrics[i, j]}, R2={r2_metrics[i, j]}")

# Placeholder for num_centers and sigma
num_centers_list = [5, 10, 15]
sigma_list = [0.1, 0.5, 1.0]

# Collect results
results = []
for num_centers in num_centers_list:
    for sigma in sigma_list:
        mse = num_centers * sigma  # Replace with your actual calculation
        results.append((num_centers, sigma, mse))

# Convert results to DataFrame for plotting
results_df = pd.DataFrame(results, columns=['num_centers', 'sigma', 'mse'])

# Plot 3D for num_centers, sigma, and MSE
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(results_df['num_centers'], results_df['sigma'], results_df['mse'], c=results_df['mse'], cmap='viridis')
ax.set_xlabel('num_centers')
ax.set_ylabel('sigma')
ax.set_zlabel('MSE')
plt.title('Superficie de MSE en función de num_centers y sigma')
plt.savefig('img/num_centers_sigma_mse_3d.png')
plt.close(fig)

# Plot 2D for num_centers vs MSE colored by sigma
plt.figure(figsize=(10, 6))
scatter = plt.scatter(results_df['num_centers'], results_df['mse'], c=results_df['sigma'], cmap='viridis')
plt.colorbar(scatter, label='sigma')
plt.xlabel('num_centers')
plt.ylabel('MSE')
plt.title('MSE en función de num_centers coloreado por sigma')
plt.grid(True)
plt.savefig('img/num_centers_vs_mse_colored_by_sigma.png')
plt.close()


Resultados de las métricas:
Celda (0,0): MSE=0.1031693843989333, MAE=0.3392410073995581, R2=0.32181673543800626
Celda (0,1): MSE=0.2187379099812089, MAE=0.3090959116278682, R2=0.8934012635118701
Celda (0,2): MSE=0.7021265227293026, MAE=0.4687035754246477, R2=0.1942915490504492
Celda (0,3): MSE=0.6517873913926304, MAE=0.6204870789808062, R2=0.6790962013636508
Celda (0,4): MSE=0.8499204683623448, MAE=0.7594196354789832, R2=0.04132722524841825
Celda (0,5): MSE=0.7095981991796123, MAE=0.5286811337000699, R2=0.2971347934615084
Celda (0,6): MSE=0.8475615598985801, MAE=0.9814862748059371, R2=0.1791139229251345
Celda (0,7): MSE=0.46685649276373775, MAE=0.4009729989461408, R2=0.7807765189608346
Celda (0,8): MSE=0.30963885301398075, MAE=0.7037568837941226, R2=0.6824325355258873
Celda (0,9): MSE=0.04587825502032239, MAE=0.16215058424151263, R2=0.1317747381449046
Celda (1,0): MSE=0.31972434576518904, MAE=0.9088951864290138, R2=0.2586106882638921
Celda (1,1): MSE=0.9162309266631242, MAE=0.119063896